In [ ]:
from strikepoint.driver import LeptonDriver
from IPython.display import Image, display
from picamera2 import Picamera2

import cv2
import numpy as np

driver = LeptonDriver()

In [ ]:
def showFrame(frame):
    ok, buf = cv2.imencode('.png', frame)
    if not ok:
        raise RuntimeError("Could not encode frame to PNG")
    display(Image(data=buf.tobytes()))

In [ ]:
frameList = list(driver.getFrame(True) for _ in range(4))
frame = np.mean(frameList, axis=0)
min, max = frame.min(), frame.max()
print(f"Frame min: {min}, max: {max}")

frame = driver.getFrame(True)
outFrame = cv2.resize(frame, (frame.shape[1]*8, frame.shape[0]*8),
                      interpolation=cv2.INTER_NEAREST_EXACT)
outFrame = cv2.normalize(
    outFrame, None, 0, 65535, cv2.NORM_MINMAX).astype(np.uint16)
showFrame(outFrame)
outFrame = cv2.GaussianBlur(outFrame, (33, 33), 0)
showFrame(outFrame)

# _, thresh = cv2.threshold(outFrame, 220, 255, cv2.THRESH_BINARY)
# contours, _ = cv2.findContours(
#     thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

outFrame = cv2.normalize(
    outFrame, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
outFrame = cv2.applyColorMap(outFrame, cv2.COLORMAP_HOT)
# cv2.drawContours(outFrame, contours, -1, (0, 255, 0), 2)

showFrame(outFrame)

In [ ]:
# Import necessary libraries
from picamera2 import Picamera2
from IPython.display import display, Image, clear_output
import io
from PIL import Image as PILImage

# Initialize the camera
with Picamera2() as picam2:
    picam2.start()  # Start camera hardware

    # Capture a single image
    frame = picam2.capture_array()  # Returns a NumPy array
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    frame = cv2.rotate(frame, cv2.ROTATE_180)  # Adjust orientation if needed
    showFrame(frame)